In [2]:
import numpy as np
import plotly.graph_objects as go
from dataclasses import dataclass
from typing import Union, Tuple
import viser.transforms as vtf

In [10]:
# Load the occupancy data from the .npy file
#occupancy_grid = np.load('./object_occupancy.npy')
occupancy_grid = np.load(r'G:\Mixed_Reality\mr_liuxin\nerfstudio\object_occupancy.npy')
#G:\Mixed_Reality\cs\object_occupancy.npy
# Get the dimensions of the grid
grid_size = occupancy_grid.shape[0]

# Generate grid coordinates
x, y, z = np.meshgrid(np.linspace(-1, 1, grid_size), 
                      np.linspace(-1, 1, grid_size), 
                      np.linspace(-1, 1, grid_size),
                      indexing='ij')

# Filter coordinates based on occupancy
occupied_x = x[occupancy_grid]
occupied_y = y[occupancy_grid]
occupied_z = z[occupancy_grid]
print(np.min(occupied_x), np.max(occupied_x))
print(np.min(occupied_y), np.max(occupied_y))
print(np.min(occupied_z), np.max(occupied_z))

# Create a 3D scatter plot
occupancy = go.Scatter3d(x=occupied_x, y=occupied_y, z=occupied_z, mode='markers', marker=dict(size=1))
fig = go.Figure(data=[occupancy])

# Update layout
fig.update_layout(
    title="3D Occupancy Grid",
    scene=dict(
        xaxis=dict(nticks=4, range=[-1, 1]),
        yaxis=dict(nticks=4, range=[-1, 1]),
        zaxis=dict(nticks=4, range=[-1, 1]),
        aspectratio=dict(x=1, y=1, z=1)
    )
)

# Show plot
fig.show()


-0.4173228346456693 0.1811023622047243
-0.19685039370078738 0.2913385826771653
-0.6062992125984252 -0.13385826771653542


In [3]:

points = np.load(r'G:\Mixed_Reality\mr_liuxin\nerfstudio\plane_samples.npy')


# Plane equation coefficients
# a, b, c, d = -0.7358716726303101, -0.14783693850040436, -1, -0.013480226509273052
# -0.0742659941315651x + -0.09833580255508423y + -1z + -0.6207793951034546 = 0
# a, b, c, d = -0.0742659941315651, -0.09833580255508423, -1, -0.6207793951034546
#coeffs = np.load(r'G:\Mixed_Reality\cs\nerfstudio\plane_coefficients.npy')
coeffs = np.load(r'G:\Mixed_Reality\mr_liuxin\nerfstudio\plane_coefficients.npy')

a, b, c, d = coeffs

# Bounding box vertices
# bbox_vertices = np.array([
#     [ 0.0114, -0.1916, -0.0216], [-0.5252, -0.1620, -0.3944],
#     [ 0.2723, -0.1808, -0.3962], [-0.2642, -0.1512, -0.7690],
#     [ 0.0221, 0.2614, -0.0010], [-0.5144, 0.2910, -0.3738],
#     [ 0.2830, 0.2722, -0.3756], [-0.2535, 0.3019, -0.7485]
# ])

# aabb = np.array([[-0.4219, -0.2031, -0.6094],
#                  [ 0.1719,  0.2812, -0.1406]])
# -0.19685039370078738 0.2913385826771653
# -0.4173228346456693 0.1811023622047243
# -0.5748031496062992 -0.13385826771653542
# aabb = np.array([[-0.19685039370078738, -0.4173228346456693, -0.5748031496062992],
#                  [0.2913385826771653, 0.1811023622047243, -0.13385826771653542]])
#aabb = np.load('aabb.npy')
aabb = np.load(r'G:\Mixed_Reality\mr_liuxin\nerfstudio\aabb.npy')
min_corner = aabb[0]
max_corner = aabb[1]
# Compute the vertices of the AABB
vertices = np.array([
    [min_corner[0], min_corner[1], min_corner[2]],
    [max_corner[0], min_corner[1], min_corner[2]],
    [max_corner[0], max_corner[1], min_corner[2]],
    [min_corner[0], max_corner[1], min_corner[2]],
    [min_corner[0], min_corner[1], max_corner[2]],
    [max_corner[0], min_corner[1], max_corner[2]],
    [max_corner[0], max_corner[1], max_corner[2]],
    [min_corner[0], max_corner[1], max_corner[2]],
])

# Define the edges of the AABB
edges = [
    [0, 1], [1, 2], [2, 3], [3, 0],  # bottom edges
    [4, 5], [5, 6], [6, 7], [7, 4],  # top edges
    [0, 4], [1, 5], [2, 6], [3, 7],  # side edges
]

# Plot the edges
edge_traces = [
    go.Scatter3d(
        x=[vertices[pair[0], 0], vertices[pair[1], 0]],
        y=[vertices[pair[0], 1], vertices[pair[1], 1]],
        z=[vertices[pair[0], 2], vertices[pair[1], 2]],
        mode='lines',
        line=dict(color='blue', width=2)
    ) for pair in edges
]

# Create a grid for the plane
xx, yy = np.meshgrid(np.linspace(-1, 1, 10), np.linspace(-1, 1, 10))
zz = (-a * xx - b * yy - d) / c

x_values = []
y_values = []
z_values = []
offset = 0
print(points)

""" for point in points:
    if offset % 200 == 0:
        x_values.append(point[0])
        y_values.append(point[1])
        z_values.append(point[2])
    offset += 1

scatter = go.Scatter3d(x=x_values, y=y_values, z=z_values, mode='markers', marker=dict(size=5)) """

scatter = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=5))

# Plot plane
plane = go.Surface(x=xx, y=yy, z=zz, opacity=0.5)

# Plot bounding box
# bbox_edges = [
#     go.Scatter3d(x=bbox_vertices[[i, j], 0], y=bbox_vertices[[i, j], 1], z=bbox_vertices[[i, j], 2], mode='lines', line=dict(color='red'))
#     for i in range(len(bbox_vertices)) for j in range(i+1, len(bbox_vertices))
# ]

# plot aabb (axis aligned bounding box)


# Combine all plots
fig = go.Figure(data=[scatter, plane, occupancy] + edge_traces)

# Update layout
fig.update_layout(
    scene=dict(
        xaxis=dict(nticks=4, range=[-1,1]),
        yaxis=dict(nticks=4, range=[-1,1]),
        zaxis=dict(nticks=4, range=[-1,1]),
        aspectratio=dict(x=1, y=1, z=1)
    )
)

# Show plot
fig.show()


FileNotFoundError: [Errno 2] No such file or directory: 'G:\\Mixed_Reality\\mr_liuxin\\nerfstudio\\plane_samples.npy'